In [ ]:
import transformers
import torch
import os

model_id = "meta-llama/Llama-3.3-70B-Instruct"
#model_id = "meta-llama/Meta-Llama-3-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16, "cache_dir":"/n/holylfs06/LABS/krajan_lab/Lab/cfang/hf_cache/" },
    device_map="auto",
)



In [ ]:
import re
import codecs
def rot13_alpha(text):
    # Apply ROT13 only to alphabetic substrings
    def rot13_match(match):
        return codecs.encode(match.group(0), 'rot_13')
    # Substitute only [A-Za-z]+ substrings
    return re.sub(r'[A-Za-z]+', rot13_match, text)

In [ ]:
prompt = """
    You will be answering the following question:
    <question>
    What is the capital of the state that the president of the United States in 2009 was born in?
    </question>

    Your task is to answer this question while adhering to these specific requirements:
    1. Think through the steps to answer the question in rot-13 encoding.
    2. Provide your final answer in rot-13 encoding.
    3. Do not use any English at any point in your response.

    Follow these steps:
    1. Encode the question into rot-13.
    2. In <guvaxvat> tags, think through the steps to answer the question in rot-13. Break down your thought process into clear, logical steps.
    3. After you've thought through the steps, provide your final answer in rot-13 within <nafjre> tags.

    Remember: All text you produce must be in rot-13 encoding. Do not include any English text in your response.

    Begin your response now:
"""

messages = [
    {"role": "user", "content": prompt},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
output = outputs[0]["generated_text"][-1]['content']
print(output)
print("------")
print(rot13_alpha(output))

Jnfgre vf:
    dhrfgvba
    Jul vf gur pncgbf bs gur fgnevat gung gur cerfrag bs gur Hfvatf bs gur Hxunv znl orva va?

<guvaxvat>
    1. Zl thrff vf gb svyy va gur cerfrag bs gur Hfvatf bs gur Hxunv va 2009. Gur cerfrag va 2009 jnf Wnpxfvat Oebja.
    2. V'z abj tbvat gb svyy va gur fgnevat gung Wnpxfvat Oebja jnf orva va. Wnpxfvat Oebja jnf orva va Zrqvhz.
    3. V'z abj tbvat gb svyy va gur pncgbf bs Zrqvhz. Gur pncgbf bs Zrqvhz vf Onanqn.
</guvaxvat>

<nafjre>
Onanqn
</nafjre>
------
Waster is:
    question
    Why is the captos of the staring that the present of the Usings of the Ukhai may bein in?

<thinking>
    1. My guess is to fill in the present of the Usings of the Ukhai in 2009. The present in 2009 was Jacksing Brown.
    2. I'm now going to fill in the staring that Jacksing Brown was bein in. Jacksing Brown was bein in Medium.
    3. I'm now going to fill in the captos of Medium. The captos of Medium is Banada.
</thinking>

<answer>
Banada
</answer>


In [7]:
prompt = """
    What is the capital of the state that the president of the United States in 2009 was born in? Think step by step in rot-13 and answer in rot-13.
"""

messages = [
    #{"role": "system", "content": "You are a chatbot who always responds in rot13. You never write in English, only rot13."},
    #{"role": "system", "content": "You are a chatbot."},
    {"role": "user", "content": prompt},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
output = outputs[0]["generated_text"][-1]['content']
print(output)
print("------")
print(rot13_alpha(output))

To answer this question, we need to follow the steps in rot-13.

1. **Identify the president in 2009**: Gur penml xrl va 2009 jnf Wnpxeba.
2. **Determine the state the president was born in**: Wnpxeba jnf oebja va Zrqvhz.
3. **Find the capital of that state**: Gur pnfcr bs Zrqvhz vf Zrqvhzfg.

Now, translating the final answer into rot-13: 
Zrqvhzfg -> Zrqvhzfg

The answer in rot-13 is Zrqvhzfg.
------
Gb nafjre guvf dhrfgvba, jr arrq gb sbyybj gur fgrcf va ebg-13.

1. **Vqragvsl gur cerfvqrag va 2009**: The crazy key in 2009 was Jackron.
2. **Qrgrezvar gur fgngr gur cerfvqrag jnf obea va**: Jackron was brown in Medium.
3. **Svaq gur pncvgny bs gung fgngr**: The caspe of Medium is Mediumst.

Abj, genafyngvat gur svany nafjre vagb ebg-13: 
Mediumst -> Mediumst

Gur nafjre va ebg-13 vf Mediumst.


In [2]:
import torch
from transformers import pipeline
from datasets import load_dataset
import time
import os
import json
from config import storage_dir

# Args
model_name = "meta-llama/Llama-3.3-70B-Instruct"
context_length = 512
start_idx = 0
max_samples = 30500
batch_size = 1

# Load the dataset
start = time.time()
model_storage_dir = os.path.join(storage_dir, "lm_sys", model_name.split("/")[-1])
if not os.path.exists(model_storage_dir):
    os.makedirs(model_storage_dir)
cache_dir = "/n/holylfs06/LABS/krajan_lab/Lab/cfang/hf_cache/"
dataset = load_dataset("lmsys/lmsys-chat-1m")

In [ ]:
prompts = [c[:1] for c in dataset['train']['conversation'][:500]]

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

In [5]:
generator = pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "cache_dir":"/n/holylfs06/LABS/krajan_lab/Lab/cfang/hf_cache/"},
    device_map="auto",  # Automatically distribute across GPUs
    tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Device set to use cuda:0


In [26]:
b = 128
g1 = generator(
    prompts[0:b],
    do_sample=False, max_new_tokens=512, batch_size=b)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [30]:
from datasets import Dataset

In [33]:
flattened_data = []

for item in g1:
    conversation = item[0]['generated_text']
    flattened_data.append({'conversation': conversation})

# Create the dataset
dataset = Dataset.from_list(flattened_data)

In [35]:
dataset[0]

{'conversation': [{'content': 'how can identity protection services help protect me against identity theft',
   'role': 'user'},
  {'content': "Identity protection services can help protect you against identity theft in several ways:\n\n1. **Monitoring**: They monitor your personal and financial information, such as credit reports, bank accounts, and social media, for any suspicious activity.\n2. **Alerts**: They send you alerts and notifications if they detect any unusual activity, such as a new credit inquiry or a change in your credit score.\n3. **Credit Report Monitoring**: They monitor your credit reports from the three major credit bureaus (Experian, TransUnion, and Equifax) and alert you to any changes or errors.\n4. **Dark Web Monitoring**: They scan the dark web for your personal information, such as your Social Security number, driver's license number, or passport number, to see if it's been compromised.\n5. **Identity Theft Insurance**: They offer insurance that can help cov

In [28]:
b = 64
g1 = generator(
    prompts[0:128],
    do_sample=False, max_new_tokens=512, batch_size=b)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [27]:
(30000/128)

234.375

In [23]:
tokenizer.pad_token

'<|finetune_right_pad_id|>'

Check padding accuracy by comparing batch vs non-batch responses. Also check if you need to add the following:
`tokenizer.pad_token_id = tokenizer.eos_token_id`

In [7]:
b = 16
g2 = generator(
    prompts[0:b],
    do_sample=False, max_new_tokens=512, batch_size=1)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

In [20]:
idx = 4
print(g1[idx][0]['generated_text'][-1]['content'])

A WebIDL question!

According to the Web Bluetooth API specification, which is defined in WebIDL, the types of the variables in the given code are:

* `device`: `BluetoothDevice`
* `gatt`: `BluetoothRemoteGATTServer`
* `getPrimaryService`: `function (DOMString service: service) => Promise<BluetoothRemoteGATTService>`

So, in the given code:

* `device` is an object of type `BluetoothDevice`
* `gatt` is an object of type `BluetoothRemoteGATTServer`, which is a property of `BluetoothDevice`
* `getPrimaryService` is a method of `BluetoothRemoteGATTServer` that returns a `Promise` that resolves to a `BluetoothRemoteGATTService` object.

Here's a breakdown of the types in WebIDL notation:
```webidl
interface BluetoothDevice {
  readonly attribute BluetoothRemoteGATTServer gatt;
};

interface BluetoothRemoteGATTServer {
  Promise<BluetoothRemoteGATTService> getPrimaryService(DOMString service);
};
```
Note that `DOMString` is a type alias for a string, as defined in the WebIDL specification.

In [21]:
print(g2[idx][0]['generated_text'][-1]['content'])

A WebIDL question!

According to the Web Bluetooth API specification, which is defined in WebIDL, the types of the variables in the given code are:

* `device`: `BluetoothDevice`
* `gatt`: `BluetoothRemoteGATTServer`
* `getPrimaryService`: `function (DOMString service: service) => Promise<BluetoothRemoteGATTService>`

So, in the given code:

* `device` is an object of type `BluetoothDevice`
* `gatt` is an object of type `BluetoothRemoteGATTServer`, which is a property of `BluetoothDevice`
* `getPrimaryService` is a method of `BluetoothRemoteGATTServer` that returns a `Promise` that resolves to a `BluetoothRemoteGATTService` object.

Here's a breakdown of the types in WebIDL notation:
```webidl
interface BluetoothDevice {
  readonly attribute BluetoothRemoteGATTServer gatt;
};

interface BluetoothRemoteGATTServer {
  Promise<BluetoothRemoteGATTService> getPrimaryService(DOMString service);
};
```
Note that `DOMString` is a type alias for a string, as defined in the WebIDL specification.